This notebook is for snapping of substations to lines

In [ ]:
import os
import matplotlib.pyplot as plt 
import geopandas as gpd
#import geoplot
import pandas as pd
import numpy as np
from shapely.geometry import LineString
from shapely.geometry import Point

# Africa shape data

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
africa = world.query('continent == "Africa"')
nigeriaplot = world.query('name == "Nigeria"')
#world

# Load data

In [ ]:
# pathg = os.path.realpath("data")+'/africa_all_generators.geojson' # Generators are not required in base_network
paths = os.path.realpath("data/clean")+'/africa_all_buses_clean.geojson'
pathl = os.path.realpath("data/clean")+'/africa_all_lines_clean.geojson'

# generators = gpd.read_file(pathg).set_crs(epsg=4326, inplace=True)
substations = gpd.read_file(paths).set_crs(epsg=4326, inplace=True)
lines = gpd.read_file(pathl).set_crs(epsg=4326, inplace=True)

In [ ]:
# Filter only Nigeria
lines_ng = lines[lines.loc[:,"country"] == "nigeria"].copy()
substations_ng = substations[substations.loc[:,"country"] == "nigeria"].copy()

In [ ]:
# Assign to every line a start and end point

lines_ng["bounds"] = lines_ng["geometry"].boundary # create start and end point
# splits into coordinates
lines_ng["bus0_lon"] = lines_ng["bounds"].bounds.iloc[:,0]
lines_ng["bus0_lat"] = lines_ng["bounds"].bounds.iloc[:,1]
lines_ng["bus1_lon"] = lines_ng["bounds"].bounds.iloc[:,2]
lines_ng["bus1_lat"] = lines_ng["bounds"].bounds.iloc[:,3]

lines_ng['bus_0_coors']=lines_ng["bounds"].apply(lambda mp: mp[0])
lines_ng['bus_1_coors']=lines_ng["bounds"].apply(lambda mp: mp[1])

In [ ]:
#lines_ng[lines_ng["length"] > 10000].sort_values("length").describe()
#substations_ng

# Case 1: Create bus dataframe from line.csv (not using existing substation dataframe)

### Create bus_id for each line ending

In [ ]:
bus_s = gpd.GeoDataFrame(columns = substations_ng.columns)
bus_e = gpd.GeoDataFrame(columns = substations_ng.columns)

In [ ]:
# Read information from line.csv
bus_s[["voltage", "lon", "lat", "geometry", "country"]] = lines_ng[["voltage", "bus0_lon", "bus0_lat", "bus_0_coors", "country"]] # line start points
bus_e[["voltage", "lon", "lat", "geometry", "country"]] = lines_ng[["voltage", "bus1_lon", "bus1_lat", "bus_1_coors", "country"]] # line end points
bus_all = bus_s.append(bus_e).reset_index(drop=True)

In [ ]:
# Assign index to bus_id 
bus_all.loc[:,"bus_id"] = bus_all.index

### V1. Create station_id at same bus location

- We saw that buses are not connected exactly at one point, they are usually connected to a substation "area" (analysed on maps)
- Create station_id at exactly the same location might therefore be not always correct
- Though as you can see below, it might be still sometime the case. Examples are **station 4** (2 lines with the same voltage connect at the same point) and **station 23** (4 lines with two different voltages connect at the same point)

#TODO: Filter out the generator lines - defined as going from generator to the next station which is connected to a load. Excluding generator lines make proably sense because they are not transmission expansion relevant. For now we simplify and include generator lines.


In [ ]:
# If same location/geometry make station
bus_all_v1 = bus_all
bus_all_v1["station_id"] = bus_all_v1.groupby(["lon","lat"]).ngroup()

## INFO, case nigeria
# bus_id 0:899
# station_id 0:791
# Means, only a few bus_id's at same location

In [ ]:
# Add station_id to line dataframe
n_row = int(bus_all.shape[0]/2)  # row length
lines_ng_v1 = lines_ng.reset_index(drop=True)
lines_ng_v1['bus0'] = bus_all.loc[0:(n_row-1), ["bus_id"]]
lines_ng_v1['bus1'] = bus_all.loc[(n_row):, ["bus_id"]].reset_index(drop=True)

In [ ]:
# display(lines_ng_v1)
# display(bus_all_v1)

### V2. Create station_id if a TBD area around bus location interfere